In [6]:
from extract_parquet import read_parquet,extract_file_path
import cmbagent 
from make_prompts import make_prompt, extract_python_definition
from ins import evaluate
import os

In [8]:
# Example with a single file
df = read_parquet("data/public-00000-of-00001.parquet")


In [10]:
# --- usage ---------------------------------------------------------------
i = 0                                        # whichever row you want
prompt = make_prompt(df.iloc[i])             # combined string for that row
print(prompt)

### Problem
The Hamiltonian of a three-level system is given as $H = \begin{pmatrix}
    E_a & 0 & A \\
    0 & E_b & 0 \\
    A & 0 & E_a \\
\end{pmatrix}$ where $A$ is real. The state of the system at time $t=0$ is (in this basis) $\psi(t=0) = \frac{1}{\sqrt{2}}\begin{pmatrix}1 \\
1\\
0\end{pmatrix}$ What is the expectation value of the energy at time $t$?

### Requirements
Provide the answer in the form of \texttt{python} code. Implement the following function
\begin{python}
def expectation_value(A: float, E_a:float, E_b:float, t:float) -> float:
    pass
\end{python}


In [12]:
reference_code = extract_python_definition(df.reference_implementation.iloc[i])

In [14]:
reference_code

'def expectation_value(A: float, E_a:float, E_b:float, t:float) -> float:\n    return 0.5*(E_a+E_b)'

In [16]:
results = cmbagent.one_shot(prompt,
                   max_rounds=50,
                   agent='engineer',
                   engineer_model='gpt-4.1',
                   #researcher_model='gpt-4.1',
                   #engineer_model='gemini-2.5-pro-exp-03-25',
                            
                   work_dir="./evals",
                  )

/Users/antoidicherianlonappan/Software/cmbagent_env/lib/python3.12/site-packages/autogen/agentchat/conversable_agent.py:129: UserWarning: Update function string contains no variables. This is probably unintended.
  warnings.warn("Update function string contains no variables. This is probably unintended.")


_User (to chat_manager):

### Problem
The Hamiltonian of a three-level system is given as $H = \begin{pmatrix}
    E_a & 0 & A \\
    0 & E_b & 0 \\
    A & 0 & E_a \\
\end{pmatrix}$ where $A$ is real. The state of the system at time $t=0$ is (in this basis) $\psi(t=0) = \frac{1}{\sqrt{2}}\begin{pmatrix}1 \\
1\\
0\end{pmatrix}$ What is the expectation value of the energy at time $t$?

### Requirements
Provide the answer in the form of \texttt{python} code. Implement the following function
\begin{python}
def expectation_value(A: float, E_a:float, E_b:float, t:float) -> float:
    pass
\end{python}

--------------------------------------------------------------------------------

Next speaker: engineer


>>>>>>>> USING AUTO REPLY...
             Model    agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-4.1-2025-04-14 engineer 0.01076           2261                780          3041
engineer (to chat_manager):

**Code Explanation:**

The function `expectation_value` comput

In [46]:
results.keys()

dict_keys(['chat_history', 'final_context', 'initialization_time', 'execution_time'])

In [47]:
results['execution_time']

49.165186166763306

In [20]:
filepath = os.path.join(results['final_context']['work_dir'],extract_file_path(results['final_context']['current_codebase']))

In [22]:
with open(filepath, "r", encoding="utf-8") as f:
    candidate_code = f.read()

In [24]:
candidate_code

'# filename: codebase/hamiltonian_expectation.py\nimport numpy as np\n\ndef expectation_value(A: float, E_a: float, E_b: float, t: float) -> float:\n    r"""\n    Compute the expectation value of the energy at time t for a three-level quantum system\n    with Hamiltonian:\n        H = [[E_a, 0,   A],\n             [0,   E_b, 0],\n             [A,   0,   E_a]]\n    and initial state:\n        psi(0) = (1/sqrt(2)) * [1, 1, 0]^T\n\n    Parameters\n    ----------\n    A : float\n        Off-diagonal coupling (real), units: same as E_a, E_b (e.g., energy)\n    E_a : float\n        Energy of levels 1 and 3, units: energy\n    E_b : float\n        Energy of level 2, units: energy\n    t : float\n        Time at which to compute the expectation value, units: 1/(energy units) if hbar=1\n\n    Returns\n    -------\n    float\n        Expectation value of the energy at time t, units: energy\n\n    Notes\n    -----\n    - Uses natural units with hbar=1.\n    - The time evolution is computed via th

In [28]:
evaluate(reference_code,candidate_code)

╭─ code_match_with_similarity (1 sample): openai/gpt-4.1-2025-04-14 ──────────────────────────────────────────────╮
│ reference: def expectation_value(A: float, E_a:float, E_b:float, t:float) -> float:          dataset: (samples) │
│     return 0.5*(E_a+E_b), candidate: # filename: codebase/hamiltonian_expectation.py                            │
│ import numpy as np                                                                                              │
│                                                                                                                 │
│ def expectation_value(A: float, E_a: float, E_b: float, t: float) -> float:                                     │
│     r"""                                                                                                        │
│     Compute the expectation value of the energy at time t for a three-level quantum system                      │
│     with Hamiltonian:                                                                                           │
│         H = [[E_a, 0,   A],                                                                                     │
│              [0,   E_b, 0],                                                                                     │
│              [A,   0,   E_a]]                                                                                   │
│     and initial state:                                                                                          │
│         psi(0) = (1/sqrt(2)) * [1, 1, 0]^T                                                                      │
│                                                                                                                 │
│     Parameters                                                                                                  │
│     ----------                                                                                                  │
│     A : float                                                                                                   │
│         Off-diagonal coupling (real), units: same as E_a, E_b (e.g., energy)                                    │
│     E_a : float                                                                                                 │
│         Energy of levels 1 and 3, units: energy                                                                 │
│     E_b : float                                                                                                 │
│         Energy of level 2, units: energy                                                                        │
│     t : float                                                                                                   │
│         Time at which to compute the expectation value, units: 1/(energy units) if hbar=1                       │
│                                                                                                                 │
│     Returns                                                                                                     │
│     -------                                                                                                     │
│     float                                                                                                       │
│         Expectation value of the energy at time t, units: energy                                                │
│                                                                                                                 │
│     Notes                                                                                                       │
│     -----                                                                                                       │
│     - Uses natural units with hbar=1.                                                                           │
│     - The time evolution is computed via the eigen-dec

{'same_io_dynamic': 0.0, 'content_similarity': 0.1634308895972182}